### Models for image generation with diffusion

## Unet

## UNET++


## UNET MAMBA


## UNET TRANSFORMERS

## ```Difussers hugging face``` Teste de funções modulares para modelos de difusão

link: [Diffusers](https://huggingface.co/docs/diffusers/stable_diffusion)

Testar:
* Scheduller
* Noise 
* Prediction
* Models
